In [ ]:
''' PREREQUISITES:
    - Get an environment variable called GOOGLE_CREDS with the json of credentials
        used to access google API (cf. 1Password)
    - Run pip install all the packages listed in the import section if not already done 
        * pip install google 
        * pip install googleapiclient
'''

In [1]:
''' METHODS:
    - get_dataframe(spreadsheet_id, worksheet_name, range_name, headers=1)
    - write_dataframe(df, spreadsheet_id, worksheet_name, row=1, col=1)
    - clear_worksheet(spreadsheet_id, worksheet_name)
'''

' METHODS:\n    - get_dataframe(spreadsheet_id, worksheet_name, range_name, headers=1)\n    - write_dataframe(df, spreadsheet_id, worksheet_name, row=1, col=1)\n    - clear_worksheet(spreadsheet_id, worksheet_name)\n'

In [2]:
import os.path
import pandas as pd 
import json

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

class GoogleSheetData:
    """ Class that handles data requests about google sheets
    """

    def __init__(self):
        """ Connects to gsheet using env variable GOOGLE_CREDS (1st time connecting)
        or local file token.json if exists (subsequent connections).    
        """

        credentials = None
        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists("token.json"):
            credentials = Credentials.from_authorized_user_file("token.json", SCOPES)

        # If there are no (valid) credentials available, let the user log in.
        if not credentials or not credentials.valid:
            if credentials and credentials.expired and credentials.refresh_token:
                credentials.refresh(Request())
            else:
                config = json.loads(os.environ['GOOGLE_CREDS'])
                flow = InstalledAppFlow.from_client_config(
                        config, SCOPES)

                credentials = flow.run_local_server(port=0)

            # Save the credentials for the next run
            with open("token.json", "w") as token:
                token.write(credentials.to_json())

        self.creds = credentials
        return print('Connected to GSheets')


    def get_dataframe(self, spreadsheet_id, worksheet_name, range_name, headers=1):
        """ Returns pandas dataframe from a spreadsheet.

            :param spreadsheet_id:    ID of the spreadhsheet, found in its URL: str
            :param worksheet_name:    name of worksheet e.g. "Sheet1": str
            :param range_name:        range to take into account e.g. "A1:E15": str
            :param headers:           whether selection has headers (1) or not (0). Default 1: int

            :return:                  pandas dataframe of the selection: df
        """

        total_range = worksheet_name + '!' + range_name

        try:
            service = build("sheets", "v4", credentials=self.creds)

            # Call the Sheets API
            sheet = service.spreadsheets()
            result = (
                sheet.values()
                .get(spreadsheetId=spreadsheet_id, range=total_range)
                .execute()
            )
            values = result.get("values", [])

            if not values:
                print("No data found.")
                return
            
            if headers > 0:
                df = pd.DataFrame(values[1:], columns=values[0])
            else:
                df = pd.DataFrame(values)

            return df

        except HttpError as err:
            print(err)


    def write_dataframe(self, df, spreadsheet_id, worksheet_name, row=1, col=1):
        """ Writes a dataframe into a google worksheet.

            :param df:                  pandas dataframe to write to GSheet: df
            :param spreadsheet_id:      ID of the spreadhsheet, found in its URL: str
            :param worksheet_name:      name of worksheet, e.g. "Sheet1": str
            :param row:                 row number where df is going to be written, default 1: int
            :param col:                 column number where df is going to be written, default 1: int
            
            :return:                    dict of spreadsheet & range that has been updated, with which cells have been written on: dict
        """

        col_letter_start = chr(ord('@') + col) # Transforms a column nb into a letter, e.g. 2 -> B
        row_number_start = row

        range_name = col_letter_start + str(row_number_start)
        total_range = worksheet_name + '!' + range_name

        try:
            service = build("sheets", "v4", credentials=self.creds)

            col_names = [df.columns.values.tolist()]
            values = col_names + df.values.tolist()

            body = {"values": values}

            result = (
                service.spreadsheets()
                .values()
                .update(
                    spreadsheetId=spreadsheet_id,
                    range=total_range,
                    valueInputOption="USER_ENTERED",
                    body=body,
                )
                .execute()
            )
            return result
        
        except HttpError as error:
            print(f"An error occurred: {error}")
            return error
        

    def clear_worksheet(self, spreadsheet_id, worksheet_name):
        """ Clears data from a worksheet.

            :param spreadsheet_id:      ID of the spreadhsheet, found in its URL: str
            :param worksheet_name:      name of worksheet, e.g. "Sheet1": str

            :return:                    dict of spreadsheet & range that has been cleared: dict
        """

        try:
            service = build("sheets", "v4", credentials=self.creds)

            result = (
                service.spreadsheets()
                .values()
                .clear(
                    spreadsheetId=spreadsheet_id,
                    range=worksheet_name,
                )
                .execute()
            )
            return result
        
        except HttpError as error:
            print(f"An error occurred: {error}")
            return error

In [ ]:
gs = GoogleSheetData()

# Get dataframe 
df = gs.get_dataframe("1R3Bw9aUNvJm-1HxPTYLJyDPT9Vt0mINwQI2bq1HMfxA", "Sheet1", "A1:B100")

# Write dataframe 
gs.write_dataframe(df, "1R3Bw9aUNvJm-1HxPTYLJyDPT9Vt0mINwQI2bq1HMfxA", "Sheet2")

# Clear worksheet 
gs.clear_worksheet("1R3Bw9aUNvJm-1HxPTYLJyDPT9Vt0mINwQI2bq1HMfxA", "Sheet2")

In [ ]:
df = pd.read_csv('/Users/leazurfluh/Downloads/DO NOT EDIT -Data Recon 23 & 24/2023 CSV Files/2023_bumble_marketing_spend_editJan24_no_meta_college_refactored.csv')


In [3]:
path_files_2023 = '/Users/leazurfluh/Downloads/DO NOT EDIT -Data Recon 23 & 24/refactored_files/2023/'
path_files_2024 = '/Users/leazurfluh/Downloads/DO NOT EDIT -Data Recon 23 & 24/refactored_files/2024/'
suffix = '_refactored.csv'

In [ ]:
df_dict = {
    '2023_bumble_marketing_spend_editJan24_no_meta_college': path_files_2023,
    'Restated & Net New MMM Data - College Ambassador 2023': path_files_2023,
    '2023 Meta Restated': path_files_2023,
    '2024 Meta Restated': path_files_2024,
    'AB Updated MMM H1 2024 US Data - NYT': path_files_2024,
    'AB Updated MMM H1 2024 US Data - Snapchat': path_files_2024,
    'AB Updated MMM H1 2024 US Data - Vox': path_files_2024,
    'AB Updated MMM H1 2024 US Data - YouTube via MiQ': path_files_2024,
    'AB Updated MMM H1 2024 US Data - iHeart': path_files_2024,
    'US_Bumble_SPAN_2024 AP': path_files_2024,
    'US_Bumble_Dating Sunday_January 2024': path_files_2024,
    'US_Bumble_Audio_Q2 2024': path_files_2024,
    'US_Bumble_Anything Goes_Dating Exclusivity_2024 AP': path_files_2024,
    'IAT_Bumble_Q4_US_2024_MM': path_files_2024,
    'Havas_Bumble_Q3_US_2024_MM': path_files_2024,
    'Aug-Dec 2024 Havas Meta': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - TikTok': path_files_2024,
    'Restated & Net New MMM Data - TikTok (1)': path_files_2024,
    'AB Updated MMM H1 2024 US Data - TikTok': path_files_2024,
    'AB Updated MMM H1 2024 US Data - Google (YouTube)': path_files_2024,
    'AB Updated MMM H1 2024 US Data - CTV via MiQ (thru 6_30)': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - YouTube via Miq': path_files_2024,
    'Restated & Net New MMM Data - Meme': path_files_2024,
    'AB Updated MMM Q4 2024 US Data -  Oct OOH': path_files_2024,
    'Restated & Net New MMM Data - Influencer': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December CTV via MiQ': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - Uber': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Oct YouTube via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Oct TikTok': path_files_2024,
    'Restated & Net New MMM Data - City Marketing': path_files_2024,
    'Restated & Net New MMM Data - Pinterest': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - OOH': path_files_2024,
    'AB Updated MMM Q3 2024 US Data - CTV via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov YouTube via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December YouTube via MiQ': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov TikTok': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December TikTok': path_files_2024,
    'Restated & Net New MMM Data - Culture': path_files_2024,
    'Restated & Net New MMM Data - College Ambassador 2024': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December Snapchat': path_files_2024,
    'Restated & Net New MMM Data - Vox': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov CTV via MiQ': path_files_2024,
    'Restated & Net New MMM Data - OOH': path_files_2024,
    'Restated & Net New MMM Data - Live Nation (OOH)': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - December OOH': path_files_2024,
    'AB Updated MMM Q4 2024 US Data - Nov OOH': path_files_2024,
}

In [ ]:
df1 = pd.read_csv(path_files_2023 + '2023_bumble_marketing_spend_editJan24_no_meta_college' + suffix)
df2 = pd.read_csv(path_files_2024 + '2024 Meta Restated' + suffix)
df3 = pd.read_csv(path_files_2023 + 'Restated & Net New MMM Data - College Ambassador 2023' + suffix)
df4 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - NYT' + suffix)
df5 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - Snapchat' + suffix)
df6 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - Vox' + suffix)
df7 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - YouTube via MiQ' + suffix)
df8 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - iHeart' + suffix)
df9 = pd.read_csv(path_files_2023 + '2023 Meta Restated' + suffix)
df10 = pd.read_csv(path_files_2024 + 'US_Bumble_SPAN_2024 AP' + suffix)
df11 = pd.read_csv(path_files_2024 + 'US_Bumble_Dating Sunday_January 2024' + suffix)
df12 = pd.read_csv(path_files_2024 + 'US_Bumble_Audio_Q2 2024' + suffix)
df13 = pd.read_csv(path_files_2024 + 'US_Bumble_Anything Goes_Dating Exclusivity_2024 AP' + suffix)
df14 = pd.read_csv(path_files_2024 + 'IAT_Bumble_Q4_US_2024_MM' + suffix)
df15 = pd.read_csv(path_files_2024 + 'Havas_Bumble_Q3_US_2024_MM' + suffix)
df16 = pd.read_csv(path_files_2024 + 'Aug-Dec 2024 Havas Meta' + suffix)
df17 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - TikTok' + suffix)
df18 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - TikTok (1)' + suffix)
df19 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - TikTok' + suffix)
df20 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - Google (YouTube)' + suffix)
df21 = pd.read_csv(path_files_2024 + 'AB Updated MMM H1 2024 US Data - CTV via MiQ (thru 6_30)' + suffix)
df22 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - YouTube via Miq' + suffix)
df23 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Meme' + suffix)
df24 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data -  Oct OOH' + suffix)
df25 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Influencer' + suffix)
df26 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December CTV via MiQ' + suffix)
df27 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - Uber' + suffix)
df28 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Oct YouTube via MiQ' + suffix)
df29 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Oct TikTok' + suffix)
df30 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - City Marketing' + suffix)
df31 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Pinterest' + suffix)
df32 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - OOH' + suffix)
df33 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q3 2024 US Data - CTV via MiQ' + suffix)
df34 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov YouTube via MiQ' + suffix)
df35 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December YouTube via MiQ' + suffix)
df36 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov TikTok' + suffix)
df37 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December TikTok' + suffix)
df38 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Culture' + suffix)
df39 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - College Ambassador 2024' + suffix)
df40 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December Snapchat' + suffix)
df41 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Vox' + suffix)
df42 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov CTV via MiQ' + suffix)
df43 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - OOH' + suffix)
df44 = pd.read_csv(path_files_2024 + 'Restated & Net New MMM Data - Live Nation (OOH)' + suffix)
df45 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - December OOH' + suffix)
df46 = pd.read_csv(path_files_2024 + 'AB Updated MMM Q4 2024 US Data - Nov OOH' + suffix)




In [ ]:
df13.to_csv(path_files_2024 + 'US_Bumble_Anything Goes_Dating Exclusivity_2024 AP' + suffix, index=False)
df12.to_csv(path_files_2024 + 'US_Bumble_Audio_Q2 2024' + suffix, index=False)
df11.to_csv(path_files_2024 + 'US_Bumble_Dating Sunday_January 2024' + suffix, index=False)
df10.to_csv(path_files_2024 + 'US_Bumble_SPAN_2024 AP' + suffix, index=False)

In [ ]:

def df_transfos(df_dict):
    df_spend_by_media_quarter = pd.DataFrame(columns=['Channel', 'quarter_year', 'Spend'])
    for key in df_dict:
        df = pd.read_csv(df_dict[key] + key + suffix)
        df.columns = ['Channel', 'Country', 'DMA', 'Campaign', 'Quarter', 'Date', 'Impressions', 'Spend']
        df['source_csv'] = key
        # print(df.head())
        df.Date = df.Date.astype('datetime64[ns]')
        df['quarter_nb'] = df.Date.dt.quarter.astype(str)
        df['year_nb'] = df.Date.dt.year.astype(str)
        df['quarter_year'] = df['year_nb'] + '-' + df['quarter_nb']

        df_spend_temp = df.groupby(['source_csv', 'Channel', 'quarter_year'], as_index = False)['Spend'].sum()

        df_spend_by_media_quarter = pd.concat([df_spend_by_media_quarter, df_spend_temp], ignore_index=True)
        
    return df_spend_by_media_quarter

df_spend_grouped = df_transfos(df_dict)

In [ ]:
gs = GoogleSheetData()
gs.write_dataframe(df_spend_grouped, "1JjhkNomf5k3GFmrGw0E8Elz2bLyIKrerqTW9Rq0Fke4", "Python_Script")

In [5]:
df47 = pd.read_csv(path_files_2024 + 'Net New 2024 MMM Spend - Net New Items.csv')

In [7]:
df47.head()

,Channel,Country,DMA,Campaign,Quarter,Date,Impressions,Spend
0,Sports Sponsorships,US,NYC,Liberty Parade,Q4,2024-10-14,0,"318,099"
1,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-20,0,"25,000"
2,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-22,0,"25,000"
3,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-08-24,0,"25,000"
4,Sports Sponsorships,US,NYC,NY Liberty,Q3,2024-09-08,0,"25,000"


In [10]:
df47.dtypes

Channel         object
Country         object
DMA             object
Campaign        object
Quarter         object
Date            object
Impressions      int64
Spend          float64
dtype: object

In [ ]:
import datetime
df1.Date2 = df1.Date2.astype('datetime64[ns]')
df1['year'] = df1.Date2.dt.year.astype(str)
df_test = df1[df1['year']=='2024']
# df_test['test'] = df_test['Date2'] - datetime.timedelta(days=366)
df_test

In [9]:
df47['Spend'] = df47['Spend'].astype("str")
df47['Spend'] = df47['Spend'].str.replace('$', '')
df47['Spend'] = df47['Spend'].str.replace(',', '')
df47['Spend'] = df47['Spend'].str.replace('"', '')
df47['Spend'] = df47['Spend'].str.replace(' -', '0')
df47['Spend'] = df47['Spend'].astype("float")
# df_final['Impressions'] = df_final['Impressions'].str.replace(',', '')
# df_final['Impressions'] = df_final['Impressions'].str.replace('"', '')
# df_final['Impressions'] = df_final['Impressions'].str.replace('-', '0')
# df_final['Impressions'] = df_final['Impressions'].str.replace('.00', '')
# df_final['Impressions'] = df_final['Impressions'].fillna('0')
# df_final['Impressions'] = df_final['Impressions'].astype("int")

In [11]:
df47['Spend'].sum()

np.float64(4626434.0)

In [12]:
df47.to_csv(path_files_2024 + 'Net New 2024 MMM Spend - Net New Items' + suffix, index=False)